# Test Azure GPT-4 based Top 500 Bioconductor package

In [1]:
import os, openai, requests
    
## Declare keys
api_key = os.getenv("OPENAI_API_KEY")
search_key = os.getenv("OPENAI_SEARCH_KEY")
endpoint = os.getenv("OPENAI_ENDPOINT")

In [2]:

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

# Azure OpenAI setup
openai.api_base = endpoint
openai.api_key = api_key 
deployment_id = "gpt-4-test" 

# Azure AI Search setup
search_endpoint = "https://hmsaisearch.search.windows.net"; # Add your Azure AI Search endpoint here
search_index_name = "bioc-top-500"; # Add your Azure AI Search index name here
    
# https://hms-it-openai.openai.azure.com/openai/deployments/gpt-4-test/extensions/chat/completions?api-version=2024-02-15-preview

In [3]:
# https://hms-it-openai.openai.azure.com/openai/deployments/gpt-4-test/extensions/chat/completions?api-version=2024-02-15-preview

client = openai.AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment_id}/extensions",
    api_key=api_key,
    api_version="2023-08-01-preview",
)

In [4]:
def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
 
    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

     def send(self, request, **kwargs):
         request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
         return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

In [17]:
def ask_question(question):

    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": question}],
        model=deployment_id,
        extra_body={
            "dataSources": [
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "key": search_key,
                        "indexName": search_index_name,
                        "roleInformation": "Act as an expert in the R programming language and the Bioconductor suite of packages.  ​\n\nYour job is to advise users on the usage of the various Bioconductor packages considering the documents you have in the data store.  ​\n\nTo complete this task, you can use the data you have stored that contain the vignettes of all the packages in Bioconductor and all the reference files of every function in every package of Bioconductor. ​You may also answer some general R, general programming, or Biomedical information.\n\nIf you do not know the answer ask the user to refer to https://bioconductor.org. \n\nAdd a disclaimer at the end of each response saying this model works only on the top 500 most used Bioconductor packages."
                    }
                }
            ]
        }
    )

    print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")

    return

In [16]:
# `context` is in the model_extra for Azure
print(f"\nContext: {completion.choices[0].message.model_extra['context']['messages'][0]['content']}")


Context: {"citations": [{"content": "1\n\n2\n\nSummarizedExperiment for\nCoordinating Experimental\nAssays, Samples, and Regions of\nInterest\nMartin Morgan, Valerie Obenchain, Jim Hester, Hervé Pagès\n\nRevised: 3 May, 2018\n\nContents\n\n1 Introduction\n2 Anatomy of a SummarizedExperiment\n\n2.1 Assays\n2.2 ‘Row’ (regions-of-interest) data\n2.3 ‘Column’ (sample) data\n2.4 Experiment-wide metadata\n\n3 Constructing a SummarizedExperiment\n4 Common operations on SummarizedExperiment\n\n4.1 Subsetting\n4.2 Getters and setters\n4.3 Range-based operations\n\n5 Interactive visualization\n6 Session information\n\nIntroduction\nThe SummarizedExperiment  class is used to store rectangular matrices of experimental results,\nwhich are commonly produced by sequencing and microarray experiments. Note that\nSummarizedExperiment  can simultaneously manage several experimental results or assays  as\n\nlong as they be of the same dimensions.\n\nEach object stores observations of one or more samples,

In [ ]:
## summarized experiment 
question = "How many classes are there in the Summarized Experiment package? Just give me a number."

ask_question(question)

In [19]:
## DESeq2

question = "DESeq2 performs normalization by estimating size factors for each sample. If your experiment has 5 samples, how many size factors will DESeq2 estimate?"

ask_question(question)

assistant: DESeq2 estimates one size factor for each sample in your experiment. Therefore, if your experiment has 5 samples, DESeq2 will estimate 5 size factors [doc1][doc2].

Please note that this information is based on the top 500 most used Bioconductor packages. For more specific or detailed information, please refer to the official Bioconductor documentation at https://bioconductor.org.


In [21]:
## Limma
question = "You use limma to analyze RNA-seq data from a case-control study with 30 control samples and 30 case samples. After fitting the linear model, how many coefficients will be estimated by limma for the gene expression data (assuming no additional covariates are included in the model)?"

ask_question(question)

assistant: In the context of a case-control study with 30 control samples and 30 case samples, when you fit a linear model using the limma package in R, you will estimate two coefficients. 

The first coefficient is the intercept, which represents the baseline level of gene expression in the reference group (usually the control group). The second coefficient is the slope, which represents the difference in gene expression between the case group and the control group. 

This is under the assumption that no additional covariates are included in the model. If additional covariates were included, there would be an additional coefficient estimated for each covariate [doc3].

Please note that this model works only on the top 500 most used Bioconductor packages. For more specific information, please refer to https://bioconductor.org.
